In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
from googletrans import Translator
import json
questions_answers = json.load(open('data.json'))

def translate_text(text, target_lang='id'):
    translator = Translator()
    translation = translator.translate(text, src='en', dest=target_lang)
    return translation.text

def translate_questions_answers(qa_list):
    translated_qa_list = []
    for qa in qa_list:
        translated_qa = {
            "question": translate_text(qa["question"]),
            "answer": translate_text(qa["answer"])
        }
        translated_qa_list.append(translated_qa)
    return translated_qa_list

translated_questions_answers = translate_questions_answers(questions_answers)

In [ ]:
import csv
csv_filename = "translated_questions_answers.csv"
csv_fields = ["question", "answer"]

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_fields)
    writer.writeheader()
    for qa in translated_questions_answers:
        writer.writerow(qa)

print(f"CSV file '{csv_filename}' has been successfully created.")

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('csv', data_files='translated_questions_answers.csv')

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['question'], examples['answer'], truncation=True, padding='max_length')

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Load pre-trained BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('bert-base-multilingual-cased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define a compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    return {'accuracy': (predictions == labels).mean().item()}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    # eval_dataset=tokenized_dataset['validation'],  # Use 'validation' for evaluation
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()